# S2S Zonal statistics

Zonal statistics are run on the standardized [H3 grid](https://h3geo.org/docs/core-library/restable/); the process is run on a country-by-country basis.

For the zonal statistics, each zonal statistic is run against the source dataset as a whole, then it is stratified by urban classification from the European Commission - [GHS-SMOD](https://ghsl.jrc.ec.europa.eu/ghs_smod2019.php). This creates an summary dataset that has the standard zonal stats columns (SUM, MEAN, MAX, MIN) as well as the same for urban areas (SUM_urban, MEAN_urban, MAX_urban, MIN_urban).

In [1]:
import sys, os, importlib, math, multiprocessing
import rasterio, geojson

import pandas as pd
import geopandas as gpd
import numpy as np

from h3 import h3
from tqdm import tqdm
from shapely.geometry import Polygon

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
#import GOSTRocks.mapMisc as mapMisc
from GOSTRocks.misc import tPrint

sys.path.append("../src")
import h3_helper
import country_zonal
import global_zonal

%load_ext autoreload
%autoreload 2

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [ ]:
ntl_files = ntl.aws_search_ntl()
ntl_files

In [8]:
h3_level = 6

admin_bounds = "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM2.shp"
out_folder = f"/home/wb411133/projects/Space2Stats/Population"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
global_urban = "/home/public/Data/GLOBAL/GHSL/SMOD/GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"

## Run analysis on population by gender and age

In [ ]:
population_folder = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/GLOBAL_1km_Demographics"
pop_files = [os.path.join(population_folder, x) for x in os.listdir(population_folder) if x.endswith("1km.tif")]

In [ ]:
# get a list of h3 levels to process
h3_0_list = h3_helper.generate_lvl0_lists(6, return_gdf=True, buffer0=False)

In [ ]:
AWS_S3_BUCKET = 'wbg-geography01'
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

def run_zonal(gdf, cur_raster_file, out_file):
    cName = f'{os.path.basename(os.path.dirname(out_file))}-{os.path.basename(cur_raster_file)}'
    tPrint(f'Starting {cName}')
    res = rMisc.zonalStats(gdf, cur_raster_file, minVal=0)
    res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
    res['id'] = gdf['shape_id'].values
    res.to_csv(
        f"s3://{AWS_S3_BUCKET}/{out_file}",
        index=False,
        storage_options={
            "key": AWS_ACCESS_KEY_ID,
            "secret": AWS_SECRET_ACCESS_KEY,
            "token": AWS_SESSION_TOKEN,
        },
    )
    #res.to_csv(out_file)
    tPrint(f'**** finished {cName}')
    return(res)
    

In [ ]:

# set up mp arguments
arg_list = []
processed_list = []
keep_processing = True
while keep_processing:
    arg_list = []
    processed_list = []
    for h3_0_key, cur_gdf in h3_0_list.items():
        for pop_file in pop_files:
            filename = os.path.basename(f'{pop_file.replace(".tif", "")}_zonal.csv')
            out_s3_key = f'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/{h3_0_key}/{filename}'
            full_path = os.path.join("s3://", AWS_S3_BUCKET, out_s3_key)        
            try:
                tempPD = pd.read_csv(full_path)
                processed_list.append(filename)
            except:
                arg_list.append([cur_gdf, pop_file, out_s3_key])
    keep_processing = len(arg_list) != 0
    tPrint(f'Remaining: {len(arg_list)}\t Processed: {len(processed_list)}')


In [ ]:
for args in arg_list:
    run_zonal(*args)

In [ ]:
with multiprocessing.Pool(processes=min([70,len(arg_list)])) as pool:
    results = pool.starmap(run_zonal, arg_list[:5])

# Aggregating results to admin boundaries

In [9]:
inA = gpd.read_file(admin_bounds)

In [3]:
all_files = global_zonal.get_global_table_from_s3("WorldPop_2020_Demographics", read_data=False)
all_files

{'global_f_0_2020_1km_zonal': ['Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/8001fffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/8003fffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/8005fffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/8007fffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/8009fffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/800bfffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/800dfffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/800ffffffffffff/global_f_0_2020_1km_zonal.csv',
  'Space2Stats/h3_stats_data/GLOBAL/WorldPop_2020_Demographics/8011fffffffffff/glob

In [5]:
bucket='wbg-geography01'
prefix='Space2Stats/h3_stats_data/GLOBAL'
for key, values in all_files.items():
    all_pd = [pd.read_csv(f"s3://{bucket}/{val}") for val in values]
    cur_pd = pd.concat(all_pd)
    break

In [6]:
cur_pd.shape

(14117882, 5)

In [7]:
cur_pd.head()

,SUM,MIN,MAX,MEAN,id
0,-1.0,-1.0,-1.0,-1.0,8600504efffffff
1,-1.0,-1.0,-1.0,-1.0,860172317ffffff
2,-1.0,-1.0,-1.0,-1.0,86001d0cfffffff
3,-1.0,-1.0,-1.0,-1.0,8600406b7ffffff
4,-1.0,-1.0,-1.0,-1.0,86005e6f7ffffff


In [10]:
inA.head()

,ISO_A3,ISO_A2,WB_A3,HASC_0,HASC_1,HASC_2,GAUL_0,GAUL_1,GAUL_2,WB_REGION,...,NAM_1_WIKI,NAM_2,NAM_2_GAUL,NAM_2_STAT,NAM_2_SRCE,NAM_2_NTVE,NAM_2_WIKI,Shape_Leng,Shape_Area,geometry
0,AFG,AF,AFG,AF,AF.BD,AF.BD.AK,1,0,0,SAR,...,None,Arghanj Khwa,None,Arghanj Khwa,Arghanj Khwa,ارغنچخواه,None,1.617679,0.074255,"POLYGON ((70.77120 37.49522, 70.77751 37.50151..."
1,AFG,AF,AFG,AF,AF.BD,AF.BD.AR,1,0,0,SAR,...,None,Argo,None,Argo,Argo,ارگو,None,1.977449,0.106834,"POLYGON ((70.10776 37.01419, 70.10744 37.01628..."
2,AFG,AF,AFG,AF,AF.BD,AF.BD.BA,1,0,0,SAR,...,None,Baharak,None,Baharak,Baharak,بهارک,None,0.894333,0.032766,"POLYGON ((70.77153 37.05601, 70.77924 37.06589..."
3,AFG,AF,AFG,AF,AF.BD,AF.BD.DY,1,0,0,SAR,...,None,Darayem,None,Darayim,Darayem,درایم,None,1.295426,0.056675,"POLYGON ((70.65071 36.86746, 70.65227 36.86330..."
4,AFG,AF,AFG,AF,AF.BD,AF.BD.DA,1,0,0,SAR,...,None,Darwaz,None,Darwaz,Darwaz,درواز,None,1.888940,0.125982,"POLYGON ((70.99066 38.49031, 70.99164 38.48937..."


In [14]:
selA.geom_type.iloc[0]

'Polygon'

In [24]:
# run zonal calculation
selA = inA.loc[inA['ISO_A3'] == 'KEN'].copy()
tPrint("Start")
pop_summ = global_zonal.connect_polygons_h3_stats(selA, cur_pd, 6, 'HASC_2', fractional_res=False)
tPrint("Start")

16:31:51	Start
17:53:46	Start


In [25]:
pop_summ

,id,SUM,MIN,MAX,MEAN
0,None,2626.414368,0.371609,71.590019,3.798485
1,None,2938.139435,1.682489,88.108627,10.663259
2,None,3672.996567,0.181900,29.441103,6.604783
3,None,3590.818108,0.064446,65.257866,5.155115
4,None,2175.816925,2.035039,60.209053,7.810934
...,...,...,...,...,...
285,None,6750.591309,2.067055,555.165039,48.538044
286,None,773.489191,0.000361,23.185417,0.834491
287,None,2549.164513,0.124819,55.275227,1.982347
288,None,1258.460025,0.000085,57.504158,0.265435
